# Лабораторная работа №1

Импортирование библиотек

In [1]:
import pandas as pd, scipy
from scipy import stats
import datetime
import math
import numpy as np
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

## Шаг 1
Скачиваем данные с сайта Rp5.ru за 2019 год. (Я взял СПб). Считываем CSV файл с данными в DataFrame и преобразоваем его (заменяем розу ветров)
Параметры:
1. **T** - Температура
2. **P0** - атмосферное давление на уровне станции
3. **P** - атмосферное давление, приведённое к среднему уровню
4. **U** - Относительная влажность (%) на высоте 2 метра
5. **DD** - направление ветра
6. **ff** - скорость ветра на высоте 10-12 метров, осреднённая за 10мин период
7. **ff0** - максимальное значение порыва ветра на высоте 10-12 метров, за 10мин период
8. **ff3** - максимальное значение порыва ветра на высоте 10-12 метров, за период между сроками
9. **N** - общая облачность
10. **WW** - текущая погода
11. **W1** - прошедшая погода между сроками наблюдения 1
12. **W2** - прошедшая погода между сроками наблюдения 2
13. **Tn** - минимальная температура воздуха за прошедший период (не более 12 часов)
14. **Tx** - максимальная температура воздуха за прошедший период (не более 12 часов)
15. **Cl** - Солисто-кучевые, кучевые и другие облака
16. **Nh** - количество облаков Cl, или Cm при отсутствии Cl облаков
17. **H** - ысота основания самых низких облаков
18. **Cm** - высококучеые, высокосолистые и прочие облака
19. **Ch** - Перистые, перисто-кучевые и прочие облака
20. **VV** - горизонтальная дальность видимости
21. **Td** - температура точки росы на высоте 2 метра над поерхностью замли
22. **RRR** - количество выпавших осадков
23. **tR** - период времени за который накопленно указанное количество осадков
24. **E** - состояние поверхности почвы
25. **Tg** - Минимальная температура поверхности почвы за ночь
26. **E'** - состояние почвы со снегом или измеримым ледяным покровом
27. **sss** - высота снежного покрова

In [2]:
df = pd.read_csv('piter2.csv', delimiter=';', error_bad_lines=False, index_col=False)

# Переименовываем первый стобец
df.rename(columns={'Местное время в Санкт-Петербурге':'date'}, inplace=True)

# Конверитруем строковые даты в формат datetime64[ns]
df['date'] = pd.to_datetime(df['date'])

# Сортируем по дате
df = df.sort_values(by='date')
df
print(len(df.index))

2920


In [3]:
# Переименовываем значения
df = df.replace({'DD' : {
    'Штиль, безветрие'                       : 0,
    'Ветер, дующий с северо-северо-востока'  : 22.5,
    'Ветер, дующий с северо-востока'         : 45,
    'Ветер, дующий с востоко-северо-востока' : 67.5,
    'Ветер, дующий с востока'                : 90,
    'Ветер, дующий с востоко-юго-востока'    : 112.5,
    'Ветер, дующий с юго-востока'            : 135,
    'Ветер, дующий с юго-юго-востока'        : 157.5,
    'Ветер, дующий с юга'                    : 180,
    'Ветер, дующий с юго-юго-запада'         : 202.5,
    'Ветер, дующий с юго-запада'             : 225,
    'Ветер, дующий с западо-юго-запада'      : 247.5,
    'Ветер, дующий с запада'                 : 270,
    'Ветер, дующий с западо-северо-запада'   : 292.5,
    'Ветер, дующий с северо-запада'          : 315,
    'Ветер, дующий с северо-северо-запада'   : 337.5,
    'Ветер, дующий с севера'                 : 360
}})

In [4]:
# Уберём столбцы, где количсетво пустых значений больше 40%
print('Общая сумма: {}, thresh параметр: {}'.format(len(df.index), len(df.index) * 4 / 10))
df.dropna(axis=1, thresh=len(df.index) * 4 / 10 , inplace=True)

# Так же  убираем лишние столбцы, 
# данные которых не несут информативности для данной работы 
_drop = ['N', 'Cl', 'Nh', 'Cm', 'Ch', 'tR', 'RRR', 'H', 'WW']
for a in _drop:
    df.drop(a, axis=1, inplace=True)

# print('Cm:')
# print(df['Cm'].value_counts())
# print('Cl:')
# print(df['Cl'].value_counts())
# print('Ch:')
# print(df['Ch'].value_counts())
# print('N:')
# print(df['N'].value_counts())
# print('Nh:')
# print(df['Nh'].value_counts())
# print('H:')
# print(df['H'].value_counts())
# print('WW:')
# print(df['WW'].value_counts())

print("Сумма Nan значений ДО заполнения средними значениями:")
print( df.isna().sum())

Общая сумма: 2920, thresh параметр: 1168.0
Сумма Nan значений ДО заполнения средними значениями:
date    0
T       0
Po      1
P       1
Pa      5
U       1
DD      1
Ff      0
VV      4
Td      1
dtype: int64


In [5]:
columns = list(df)
columns.remove('date')

# Заполняем средними значениями
for a in columns:
    df[a].fillna(df[a].groupby([df.date.dt.year, df.date.dt.month]).transform('mean'), inplace=True)
print( df.isna().sum())
df

date    0
T       0
Po      0
P       0
Pa      0
U       0
DD      0
Ff      0
VV      0
Td      0
dtype: int64


,date,T,Po,P,Pa,U,DD,Ff,VV,Td
2919,2018-01-01 00:00:00,0.8,755.6,756.2,-0.5,75.0,157.5,2,10.0,-3.1
2918,2018-01-01 03:00:00,0.4,754.5,755.1,-1.1,79.0,112.5,3,10.0,-2.9
2917,2018-01-01 06:00:00,0.0,752.8,753.4,-1.7,82.0,112.5,2,10.0,-2.8
2916,2018-01-01 09:00:00,-0.5,751.1,751.7,-1.7,94.0,112.5,2,4.0,-1.3
2915,2018-01-01 12:00:00,0.0,750.1,750.7,-1.0,94.0,112.5,2,2.0,-0.8
...,...,...,...,...,...,...,...,...,...,...
4,2018-12-31 09:00:00,-1.6,767.5,768.1,1.2,70.0,45.0,1,10.0,-6.4
3,2018-12-31 12:00:00,-1.8,768.4,769.0,0.9,73.0,337.5,1,10.0,-6.0
2,2018-12-31 15:00:00,-2.1,768.0,768.6,-0.4,73.0,292.5,1,10.0,-6.4
1,2018-12-31 18:00:00,-3.2,766.8,767.5,-1.2,81.0,225.0,2,10.0,-6.0


## Шаг 2
Вывод диаграмм **температуры** по месяцам

In [6]:
months = ['January', 'Febrary', 'March', 'April', 'May', 'Jule', 
          'July', 'August', 'September', 'October', 'November', 'December']
plot_value = 'T'
for month_ in months: 
    to_plot = df[['date', plot_value]]
    to_plot = to_plot[to_plot.date.dt.month == (months.index(month_) + 1)]
    to_plot.plot(title='Temerature in {}'.format(month_), x='date', y=plot_value, figsize=(9,6), grid=True)

# Шаг 3
Расчёт дисперисии, среднеквадратического отклонения и мат. ожидания

$ \bar{X} = \frac{\sum x_i} {N}$ - среднее арифметическое

$ D = \frac{\sum (x_i - \bar{x})^2} {N}$ - дисперсия

$ \sigma = \sqrt{D}$ - cреднеквадратическое отклонение

In [7]:
columns = list(df)
columns.remove('date')
for col in columns:
    printmd('### Значения для **{}**'.format(col))
    
    n = len(df[col].index)
    # Среднее арифметическое
    summ = sum(df[col])
    mean = summ / n
    print('Среднее арифметическое: {}'.format(round(mean, 3)))
    printmd('Среднее арифметическое **Pandas**: {}'.format(round(df[col].mean(), 3)))
    
    # Дисперсия
    sq_summ = sum([(x - mean)**2 for x in df[col]])
    dispersion = sq_summ / n
    print('Дисперсия {}'.format(round(dispersion, 3)))
    printmd('Дисперсия **Pandas**: {}'.format(round(df[col].var(), 3)))
    
    # Среднеквадратическое отклонение
    sq_dv = math.sqrt(dispersion)
    print('Среднеквадратическое отклонение: {}'.format(round(sq_dv, 3)))
    printmd('Среднеквадратическое отклонение **Pandas**: {}'.format(round(df[col].std(), 3)))

    printmd('---')   

### Значения для **T**

Среднее арифметическое: 7.072


Среднее арифметическое **Pandas**: 7.072

Дисперсия 112.665


Дисперсия **Pandas**: 112.703

...

Дисперсия 82.782


Дисперсия **Pandas**: 82.81

Среднеквадратическое отклонение: 9.098


Среднеквадратическое отклонение **Pandas**: 9.1

---

# Шаг 4
расчёт корреляции

$ r_{xy} = \frac {\sum (x_i - \bar{x}) * (y_i - \bar{y})} {\sqrt {\sum (x_i - \bar{x})^2 * (y_i - \bar{y})^2}}$

In [8]:
columns = list(df)
columns.remove('date')
f = 'T'
s = 'Po'
corr = pd.DataFrame(index= columns)
for f in columns:
    t = []
    for s in columns:
        summ_1 =  sum(np.array(df[f] - df[f].mean()) * np.array(df[s] - df[s].mean()))
        summ_2 = math.sqrt(sum(pow(np.array(df[f] - df[f].mean()), 2)) *
                           sum(pow(np.array(df[s] - df[s].mean()), 2)))
        t.append(summ_1 / summ_2)
    corr[f] = t
printmd("### Ручная корреляция")     
corr

### Ручная корреляция

,T,Po,P,Pa,U,DD,Ff,VV,Td
T,1.000000,-0.144068,-0.147397,-0.065869,-0.453882,0.047337,0.013610,0.157905,0.900470
Po,-0.144068,1.000000,0.999810,0.060222,-0.126077,-0.164079,-0.238574,0.114984,-0.227180
P,-0.147397,0.999810,1.000000,0.057043,-0.124522,-0.164046,-0.238201,0.114489,-0.230138
Pa,-0.065869,0.060222,0.057043,1.000000,0.061886,0.068705,-0.055147,0.070058,-0.035144
U,-0.453882,-0.126077,-0.124522,0.061886,1.000000,0.002524,-0.078835,-0.257270,-0.032583
DD,0.047337,-0.164079,-0.164046,0.068705,0.002524,1.000000,0.429084,0.011835,0.057032
Ff,0.013610,-0.238574,-0.238201,-0.055147,-0.078835,0.429084,1.000000,-0.002370,-0.011138
VV,0.157905,0.114984,0.114489,0.070058,-0.257270,0.011835,-0.002370,1.000000,0.073438
Td,0.900470,-0.227180,-0.230138,-0.035144,-0.032583,0.057032,-0.011138,0.073438,1.000000


In [9]:
printmd("### Pandas корреляция")  
df.corr() 

### Pandas корреляция

,T,Po,P,Pa,U,DD,Ff,VV,Td
T,1.000000,-0.144068,-0.147397,-0.065869,-0.453882,0.047337,0.013610,0.157905,0.900470
Po,-0.144068,1.000000,0.999810,0.060222,-0.126077,-0.164079,-0.238574,0.114984,-0.227180
P,-0.147397,0.999810,1.000000,0.057043,-0.124522,-0.164046,-0.238201,0.114489,-0.230138
Pa,-0.065869,0.060222,0.057043,1.000000,0.061886,0.068705,-0.055147,0.070058,-0.035144
U,-0.453882,-0.126077,-0.124522,0.061886,1.000000,0.002524,-0.078835,-0.257270,-0.032583
DD,0.047337,-0.164079,-0.164046,0.068705,0.002524,1.000000,0.429084,0.011835,0.057032
Ff,0.013610,-0.238574,-0.238201,-0.055147,-0.078835,0.429084,1.000000,-0.002370,-0.011138
VV,0.157905,0.114984,0.114489,0.070058,-0.257270,0.011835,-0.002370,1.000000,0.073438
Td,0.900470,-0.227180,-0.230138,-0.035144,-0.032583,0.057032,-0.011138,0.073438,1.000000


# Расчёт доверительного интервала

$ \bar{x} - \sigma < \mu < \bar{x} + \sigma $

In [10]:
columns = list(df)
columns.remove('date')
for col in columns:
    # ручной
    mean = df[col].mean()
    sq_dv = df[col].std()
    z = 1.96 # Из таблицы Z оценок
    left, right = mean - z * sq_dv / math.sqrt(len(df.index)), mean + z * sq_dv/ math.sqrt(len(df.index))
    printmd('**Ручной** расчёт для **{}**'.format(col))
    printmd('$${} < \mu < {}$$'.format(left, right))
    
    # scipy
    sk = stats.norm.interval(0.95, loc=mean, scale=sq_dv / math.sqrt(len(df.index)))
    printmd('**Scipy** расчёт для **{}**'.format(col))
    printmd('$${} < \mu < {}$$'.format(sk[0], sk[1]))
    printmd('---')

**Ручной** расчёт для **T**

$$6.686819341832544 < \mu < 7.456947781455128$$

**Scipy** расчёт для **T**

$$6.6868264174779455 < \mu < 7.456940705809727$$

---

...

**Ручной** расчёт для **Td**

$$1.473873958467831 < \mu < 2.13401523237017$$

**Scipy** расчёт для **Td**

$$1.4738800235932417 < \mu < 2.1340091672447596$$

---

In [11]:
#df.groupby([df.date.dt.year, df.date.dt.month]).T.mean().plot(title="Температура", figsize=(8,4), 
#                                                              grid=True, x_compat=True, kind='line')